In [1]:
!pip install beautifulsoup4
!pip install requests_cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# Montamos nuestro google drive en el colab

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from gensim.models import Word2Vec

In [3]:
class EpochSaver: pass

In [4]:
from pathlib import Path
DATA_HOME = Path("/content/gdrive/My Drive/ml-practico-data")

In [5]:
model = Word2Vec.load(str(DATA_HOME / 'w2v/epoch_10'))

In [46]:
from IPython.display import HTML, display
from bs4 import BeautifulSoup
import requests
import requests_cache

requests_cache.install_cache()

def get_name(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    return soup.select('.header .itemprop')[0].text

def get_image(id):
    response = requests.get(f'https://www.imdb.com/name/{id}/')
    soup = BeautifulSoup(response.content)
    candidates = soup.select('#name-poster')
    return candidates[0].attrs['src'] if candidates else 'https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB466677935_.png'

def render_person(id):
    name = get_name(id)
    picture = get_image(id)
    return f"""
    <div style="width: 150px; text-align: center">
        <h4 style='margin-top: -5px'>{name}</h4>
        <div style='font-size:75%; margin-bottom: 5px'>{id}</div>
        <a href="https://www.imdb.com/name/{id}" target="_blank">
            <img style="width: 100px; display: block; margin-left: auto; margin-right: auto;" src="{picture}"/>
        </a>
    </div>
    """

In [34]:
def show_similars(id, n=10):
    display(HTML(render_person(id)))
    renders = []
    for similar_id, score in model.wv.most_similar(id, topn=n):
        renders.append(render_person(similar_id))
        
    carousel = ''.join(
        [
            f'<div style="margin-left: 10px; float: left">{p}</div>' 
            for p in renders
        ]
    )
    display(HTML(f'<div style="width: 1800px">{carousel}</div>'))

In [35]:
show_similars('nm0001401')

In [36]:
show_similars('nm0201857')

In [37]:
show_similars('nm0006168')

In [38]:
show_similars('nm0655585')

In [39]:
show_similars('nm0000375')

In [40]:
show_similars('nm0511088')

In [41]:
show_similars('nm0000150')

# Algunos random

In [42]:
# Clonamos el repo para usar el codigo de la lib
!git clone https://github.com/elsonidoq/ml-practico-2022.git
!cd ml-practico-2022; git pull

fatal: destination path 'ml-practico-2022' already exists and is not an empty directory.
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/elsonidoq/ml-practico-2022
   7c90893..5edad55  main       -> origin/main
Updating 7c90893..5edad55
Fast-forward
 notebooks/clase-3/05-crew-embeddings-check.ipynb | 4419 +++++++++++-----------
 1 file changed, 2283 insertions(+), 2136 deletions(-)


In [43]:
import sys
sys.path.append('ml-practico-2022')

from lib import data

In [ ]:
import pandas as pd

title_basics = data.load_title_basics(DATA_HOME)
principals_df = pd.read_csv(DATA_HOME / 'title.principals.tsv.gz', compression='gzip', sep='\t')
ratings_df = pd.read_csv(DATA_HOME / 'title.ratings.tsv.gz', compression='gzip', sep='\t')

In [19]:
candidates = (
    principals_df[
        principals_df.category.isin(['actress', 'actor', 'director']) &
        principals_df.tconst.isin(set(title_basics[title_basics.startYear >= 1980].tconst)) &
        principals_df.tconst.isin(set(ratings_df[ratings_df.numVotes > 100000].tconst))
    ].nconst.unique()
)

candidates = [c for c in candidates if c in model.wv.vocab]
len(candidates)

3788

In [47]:
from random import choice

for _ in range(10):
    id = choice(candidates)
    show_similars(id)